In [70]:
import numpy as np
import glob
import os
import pandas as pd
import pickle


In [3]:
# pkl_file_path = "/home/jony/Git/GAN-N-Net/datasets/new_raw_data/site4_28_classes_ready2go.pkl"
pkl_file_path = "/home/jony/git/GAN-N-Net/datasets/new_raw_data/site4_28_classes_ready2go.pkl"

In [ ]:
# pd.read_pickle(pkl_file_path)
import dill
import pandas as pd

def read_pickle_in_chunks(file_path, chunk_size=10):
    with open(file_path, 'rb') as f:
        while True:
            try:
                chunk = dill.load(f)
                yield pd.DataFrame(chunk)
            except EOFError:
                break

# Usage
file_path = 'large_pickle_file.pkl'
chunk_size = 1000

for i, chunk in enumerate(read_pickle_in_chunks(file_path, chunk_size)):
    print(f"Chunk {i+1}:")
    print(chunk.head())

In [5]:
df = pd.read_pickle(pkl_file_path)

In [24]:
def build_port_TDL(row, min_delta=1e-5):
    to_return = []

    measures = np.array(row['udps.protocol_header_fields_enh'][:, 1])
    # sizes = (measures - measures.min()) / (measures.max() - measures.min())
    sizes = measures #  (maybe I can change all zeroes to 1. so it will be different than no data)
    directions = [0]*len(measures)
    for el_i, el in enumerate(row['udps.protocol_header_fields_enh'][:, 0]): # enumerate(np.array(hlp_protocol_header_fields_enh[:,0])): # enumerate(row['udps.protocol_header_fields_enh'][:, 0]):
        if el != 0:
            directions[el_i] = 1
            # normalized_measures[el_i] *= -1
    measures = np.array(row['udps.protocol_header_fields_enh'][:, 2])
    measures = measures / 1000
    times = [0] * len(measures)
    for i in range(1, len(times)):
        times[i] = times[i-1] + max(measures[i], min_delta)
    # if measures.max() != measures.min():    # we want to avoid 0/0
    #     times = (measures - measures.min()) / (measures.max() - measures.min())

    for item in zip(times, directions, sizes):
        to_return.append(list(item))

    return to_return

In [20]:
data_dir = "/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/"
dest_dir = 'datasets/new_raw_data/site5_1500_packets_al_to_share/csvs_ready_tdl'
# get list of all files in data_dir
files = glob.glob(data_dir + "*.csv")
print(files[:10])
files = files[:10]

['/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-14_v7_24_05_24_04_48_10_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-02_v7_24_05_24_06_25_50_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-16_v7_24_05_24_01_32_37_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-23_v7_24_05_23_14_36_53_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-07_v7_24_05_23_17_54_02_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-06_v7_24_05_23_15_42_14_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data/site5_1500_packets_al_to_share/csvs/tcp1-vmsp-p-23_v7_24_05_24_08_38_09_600_n8_WF.csv', '/home/jony/git/GAN-N-Net/datasets/new_raw_data

In [26]:
file.split('/')[-1]

'tcp1-vmsp-p-14_v7_24_05_24_04_48_10_600_n8_WF.csv'

In [34]:
# create an empty list to store the DataFrames
dfs = []

# loop through each CSV file and read it into a DataFrame
for file in files:
    df = pd.read_csv(file)
    df['filename'] = file.split('/')[-1]
    df['packet_number'] = df['udps.protocol_header_fields_enh'].str.count('\n')
    # this raw convert the nfstream plugin output from string to list of lists - for now it is crashing
    df['udps.protocol_header_fields_enh'] = df['udps.protocol_header_fields_enh'].apply(lambda x: np.array([list(map(int, row.strip().strip('[]').split())) for row in x.strip('[]').split('\n') if row.strip()]))
    df["tdl"] = df.apply(build_port_TDL, axis=1)
    df.to_csv(os.path.join(dest_dir, file.split('/')[-1]), index=False)
    # this raw convert the nfstream plugin output from string to list of lists - for now it is crashing
    # df['splited'] = df['udps.protocol_header_fields_enh'].str.replace('[', '').str.replace(']', '').apply(lambda x: [list(map(int, row.split())) for row in x.split('\n')])
    # dfs.append(df)

# concatenate all DataFrames into a single one
# big_df = pd.concat(dfs, ignore_index=True)

In [60]:
df.new_application_types.value_counts()
df[df.new_application_types!="other"].new_application_types.value_counts()

new_application_types
TikTok                  86
Youtube_YoutubeMusic    53
Instagram_Threads        4
FacebookMessenger        2
Facebook                 1
Yahoo                    1
XVideos                  1
Name: count, dtype: int64

In [69]:
df_tdl = df['tdl'].apply(lambda x: pd.DataFrame(x, columns=['col1', 'col2', 'col3']))
df_tdl["new_application_types"] = df["new_application_types"]
df_tdl.colums

AttributeError: 'Series' object has no attribute 'colums'

In [42]:
tdl_dfs.shape

(341,)

In [ ]:
# all can be found in EIMTC git repo (ask amit and chen)
# udps.protocol_header_fields_enh - nfstream plugin that contains:
    # Dirrection: 0 - source to destination, 1 - destination to source
    # payload_size: not include headers
    # delta_time: time from previus packet
    # scapy_tcp: I don't kknow what it is
    # len(scapy_tcp.options): I don't kknow what it is
    # scapy_ip.ttl: ttl value....

# These are the labels coulumns:
    # new_application_types
    # new_traffic_types

In [ ]:
# big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.split())) for row in x.strip().split('\n')])
# big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.strip('[]').split())) for row in x.strip().split('\n') if row.strip()])
# big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.strip().strip('[]').split())) for row in x.strip('[]').split('\n') if row.strip()])

# big_df.iloc[:1000]['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.strip().strip('[]').split())) for row in x.strip('[]').split('\n') if row.strip()])

In [6]:
big_df.columns

Index(['Unnamed: 0', 'bidirectional_first_seen_ms', 'new_application_types',
       'new_stnn', 'udps.packets_raw_sizes', 'udps.protocol_header_fields_enh',
       'udps.simple_tig_adj', 'udps.simple_tig_features',
       'application_category_name', 'udps.stnn_image', 'udps.OTT',
       'udps.flowType', 'new_traffic_types', 'application_name',
       'udps.n_bytes', 'filename', 'packet_number'],
      dtype='object')

In [13]:
big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: np.array([list(map(int, row.strip().strip('[]').split())) for row in x.strip('[]').split('\n') if row.strip()]))
# big_df['splited'] = big_df.apply(from_protocol_header_fields_enh_saved_to_ndarray, axis=1)

In [14]:
big_df["tdl"] = big_df.apply(build_port_TDL, axis=1)

In [19]:
big_df.tdl.iloc[0][-20:]

[[5.57275999999989, 1, 1232],
 [5.572769999999889, 1, 1232],
 [5.572779999999889, 1, 1232],
 [5.572789999999888, 1, 1232],
 [5.572799999999888, 1, 1232],
 [5.574799999999888, 1, 1232],
 [5.5748099999998875, 1, 1232],
 [5.574819999999887, 1, 1232],
 [5.574829999999887, 1, 1232],
 [5.574839999999886, 1, 1232],
 [5.5778399999998864, 1, 1232],
 [5.577849999999886, 1, 1232],
 [5.577859999999886, 1, 1232],
 [5.577869999999885, 1, 1232],
 [5.577879999999885, 1, 1232],
 [5.579879999999885, 1, 1232],
 [5.579889999999884, 1, 1232],
 [5.579899999999884, 1, 1232],
 [5.579909999999884, 1, 1232],
 [5.579919999999883, 1, 1232]]